In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [6]:
import sys
sys.path.append("..") ## resetting the path to the parent directory

In [7]:
from src.pipeline import *

In [10]:
seed_df = pd.read_csv('../data/seed_data.csv')

/Users/anotherjonlee/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (56,57,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
seed_df.columns[56:60]

Index(['firstBloodKill', 'firstBloodAssist', 'firstTowerKill',
       'firstTowerAssist'],
      dtype='object')

In [27]:
seed_df.isnull().sum().sort_values(ascending=False)[:10]

firstInhibitorAssist              780
firstInhibitorKill                780
firstTowerKill                    290
firstTowerAssist                  290
firstBloodAssist                  130
firstBloodKill                    130
physicalDamageDealtToChampions      0
pentaKills                          0
tripleKills                         0
damageSelfMitigated                 0
dtype: int64

In [19]:
def feature_separator(df):
    numeric_features = []
    cat_features = []

    for col in df.columns[3:]:
        if df[col].dtype == 'int64':
            numeric_features.append(col)
        else:
            cat_features.append(col)
    return numeric_features,cat_features

In [16]:
cat_features

['firstBloodKill',
 'firstBloodAssist',
 'firstTowerKill',
 'firstTowerAssist',
 'firstInhibitorKill',
 'firstInhibitorAssist',
 'highestAchievedSeasonTier',
 'role',
 'lane']

In [17]:
numeric_features

['item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'kills',
 'deaths',
 'assists',
 'largestKillingSpree',
 'largestMultiKill',
 'killingSprees',
 'longestTimeSpentLiving',
 'doubleKills',
 'tripleKills',
 'quadraKills',
 'pentaKills',
 'unrealKills',
 'totalDamageDealt',
 'magicDamageDealt',
 'physicalDamageDealt',
 'trueDamageDealt',
 'largestCriticalStrike',
 'totalDamageDealtToChampions',
 'magicDamageDealtToChampions',
 'physicalDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'totalHeal',
 'totalUnitsHealed',
 'damageSelfMitigated',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'visionScore',
 'timeCCingOthers',
 'totalDamageTaken',
 'magicalDamageTaken',
 'physicalDamageTaken',
 'trueDamageTaken',
 'goldEarned',
 'goldSpent',
 'turretKills',
 'inhibitorKills',
 'totalMinionsKilled',
 'neutralMinionsKilled',
 'neutralMinionsKilledTeamJungle',
 'neutralMinionsKilledEnemyJungle',
 'totalTimeCrowdControlDealt',
 'champLevel',
 'visionWardsBoughtIn